In [1]:
%run Loading_data.ipynb

In [2]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [3]:
batch = []
target = []
for j in range(500):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    batch.append(tensor_pad)
    target.append(torch.tensor(y_train[0+j*40:40+j*40]))

In [4]:
v_batch = []
v_target = []
for j in range(125):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    v_batch.append(tensor_pad)
    v_target.append(torch.tensor(y_train[0+j*40:40+j*40]))

In [27]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = nn.RNN(300,300) 
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.fc1(x)
        x = torch.max(F.relu(x[0]),1)
        x = self.fc2(x.values)
        return x

net_Rnn = Net()
print(net_Rnn)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): RNN(300, 300)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [28]:
import torch.optim as optim
optimizer = optim.Adam(net_Rnn.parameters(),betas=(0.9, 0.999), lr=0.001)

In [29]:
for epoch in range(1): 
    for j in range(500):
        outputs = net_Rnn(batch[j])
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j])
        loss.backward()
        optimizer.step()
        print(loss.item())

print('Finished Training')

TypeError: relu(): argument 'input' (position 1) must be Tensor, not tuple

In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = nn.LSTM(300,300)  
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.fc1(x)
        x = torch.max(F.relu(x[0]),1)
        x = self.fc2(x.values)
        return x

net_LSTM = Net()
print(net_LSTM)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): LSTM(300, 300)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [13]:
import torch.optim as optim
optimizer = optim.Adam(net_LSTM.parameters(),betas=(0.9, 0.999), lr=0.001)

In [14]:
for epoch in range(1): 
    for j in range(500):
        outputs = net_LSTM(batch[j])
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j])
        loss.backward()
        optimizer.step()
        print(loss.item())

print('Finished Training')

0.6914360523223877
0.7032051682472229
0.700984001159668
0.6870556473731995
0.6806427836418152
0.6819310188293457
0.6883306503295898
0.6799372434616089
0.6774981021881104


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 24134400000 bytes. Buy new RAM!


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(125):
        outputs = net_LSTM(v_batch[j])
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j]).sum().item()

print('Accuracy of the network on the 5000 test: %d %%' % (
    100 * correct / total))